In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
train_dir = 'Dataset/train'
validation_dir = 'Dataset/val'
test_dir = 'Dataset/test'

In [6]:
def get_dataset_from_directory(dire, batch_size, img_size):
    return image_dataset_from_directory(dire,shuffle=True,
                                            batch_size=batch_size,
                                            image_size=img_size)

In [12]:
def train_model(batch_size, img_size, base_learning_rate, initial_epochs, fine_tune_at, fine_tune_epochs):
    train_dataset = get_dataset_from_directory(train_dir, batch_size, img_size)
    validation_dataset = get_dataset_from_directory(validation_dir, batch_size, img_size)
    test_dataset = get_dataset_from_directory(test_dir, batch_size, img_size)

    val_batches = tf.data.experimental.cardinality(validation_dataset)

    AUTOTUNE = tf.data.experimental.AUTOTUNE

    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
    # DATA AUGMENTATION
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    ])

    preprocess_input = tf.keras.applications.densenet.preprocess_input
    IMG_SHAPE = img_size + (3,)
    base_model = tf.keras.applications.DenseNet201(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')

    image_batch, label_batch = next(iter(train_dataset))
    feature_batch = base_model(image_batch)

    base_model.trainable = False

    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(feature_batch)

    prediction_layer = tf.keras.layers.Dense(1)
    prediction_batch = prediction_layer(feature_batch_average)

    inputs = tf.keras.Input(shape=(160, 160, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    loss0, accuracy0 = model.evaluate(validation_dataset)
    print("initial loss: {:.2f}".format(loss0))
    print("initial accuracy: {:.2f}".format(accuracy0))

    history = model.fit(train_dataset,
                        epochs=initial_epochs,
                        validation_data=validation_dataset,
                        verbose=0)

    base_model.trainable = True

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate / 10),
                  metrics=['accuracy'])

    total_epochs = initial_epochs + fine_tune_epochs

    history_fine = model.fit(train_dataset,
                             epochs=total_epochs,
                             initial_epoch=history.epoch[-1],
                             validation_data=validation_dataset,
                             verbose=0)

    loss, accuracy = model.evaluate(test_dataset)
    print('Test accuracy :', accuracy)
    return accuracy, history, history_fine

In [13]:
batch_size = [16, 32, 64, 128]
IMG_SIZE = (160, 160)
base_learning_rate = [0.0001]
initial_epochs = [10, 20, 30]
fine_tune_epochs = [10, 20, 30]
fine_tune_at = [630, 640, 650, 660, 670]

for batch_sz in batch_size:
    for lr in base_learning_rate:
        for in_epochs in initial_epochs:
            for ft_epochs in fine_tune_epochs:
                for ft_at in fine_tune_at:
                    print("Hyper parameters: batch size: {} learning_rate: {} initial_epochs: {}, fine_tune_epochs: {}, fine_tune_at: {}"
                          .format(batch_sz, lr, in_epochs, ft_epochs, ft_at))
                    acc, _, _ = train_model(batch_sz, IMG_SIZE, lr, in_epochs, ft_at, ft_epochs)
                    print("Accuracy = {}".format(acc))


Hyper parameters: batch size: 16 learning_rate: 0.0001 initial_epochs: 10, fine_tune_epochs: 10, fine_tune_at: 630
Found 38391 files belonging to 2 classes.
Found 8226 files belonging to 2 classes.
Found 8227 files belonging to 2 classes.
515/515 [==============================] - 29s 56ms/step - loss: 0.7726 - accuracy: 0.4408
initial loss: 0.77
initial accuracy: 0.44
515/515 [==============================] - 29s 57ms/step - loss: 0.2518 - accuracy: 0.9369
Test accuracy : 0.9369150400161743
Accuracy = 0.9369150400161743
Hyper parameters: batch size: 16 learning_rate: 0.0001 initial_epochs: 10, fine_tune_epochs: 10, fine_tune_at: 640
Found 38391 files belonging to 2 classes.
Found 8226 files belonging to 2 classes.
Found 8227 files belonging to 2 classes.
515/515 [==============================] - 28s 53ms/step - loss: 0.8374 - accuracy: 0.3519
initial loss: 0.84
initial accuracy: 0.35
515/515 [==============================] - 29s 56ms/step - loss: 0.2172 - accuracy: 0.9367
Test accu

KeyboardInterrupt: 